# 공통함수 정의

In [ ]:
import ee
from google.oauth2.service_account import Credentials
from google.auth.exceptions import GoogleAuthError  # 올바른 범위 설정



SCOPES = ["https://www.googleapis.com/auth/earthengine"]

try:
    # credentials = Credentials.from_service_account_file(
    #     "my-first_project.json",  ## [서비스 사용량 소비자, Earth Engine 리소스 뷰어] 역할
    #     # "ee-hoony77lee-9c802931e08f.json", ## [서비스 사용량 소비자, Earth Engine 리소스 뷰어] 역할
    #     scopes=SCOPES,
    # )

    # print(f"creendtials : {credentials.service_account_email}")
    # ee.Initialize(credentials=credentials)

    # ee.Authenticate()
    # ee.Initialize(project="lively-pursuit-426306-i4")
    # ee.Initialize(project="aerobic-tesla-417706")

    credentials = Credentials.from_service_account_file(
        "aerobic-tesla-417706-7ef0ce8ab6f5.json", scopes=SCOPES
    )
    ee.Initialize(credentials=credentials)
    print("Initialization successful.")

except GoogleAuthError as e:
    print(f"Authentication failed: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

In [18]:
import cx_Oracle
import os
import logging

# 로깅 설정
logging.basicConfig(level=logging.DEBUG)
# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def get_db_connection():
    try:
        logger.info("Oracle에 연결 시도 중")
        host = os.getenv("ORACLE_HOST")
        port = os.getenv("ORACLE_PORT")
        service_name = os.getenv("ORACLE_SERVICE_NAME")
        user = os.getenv("ORACLE_USER")
        password = os.getenv("ORACLE_PASSWORD")

        logger.debug(f"연결 정보: {host}, {port}, {service_name}")

        dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
        logger.debug(f"DSN: {dsn}")

        connection = cx_Oracle.connect(user=user, password=password, dsn=dsn)
        return connection
    except cx_Oracle.Error as error:
        logger.error(f"Oracle 연결 오류: {error}")
        raise

def extract_geometry_data(geometry):
    if geometry is None:
        return None
    return {
        'SDO_GTYPE': geometry.SDO_GTYPE,
        'SDO_SRID': geometry.SDO_SRID,
        'SDO_POINT': geometry.SDO_POINT.getvalue() if geometry.SDO_POINT else None,
        'SDO_ELEM_INFO': geometry.SDO_ELEM_INFO.aslist() if geometry.SDO_ELEM_INFO else None,
        'SDO_ORDINATES': geometry.SDO_ORDINATES.aslist() if geometry.SDO_ORDINATES else None
    }

def get_roi_list():
    try:
        # Oracle 연결
        connection = get_db_connection()
        cursor = connection.cursor()

        # SQL 쿼리 실행
        query = "SELECT ROI_ID, POLYGON, DAM, RECT FROM ROI_SPATIAL"
        cursor.execute(query)

        # 결과 가져오기
        results = cursor.fetchall()

        # 결과 처리
        roi_list = []
        for row in results:
            roi_id, polygon, dam, rect = row
            roi_list.append({
                'ROI_ID': roi_id,
                'POLYGON': extract_geometry_data(polygon),
                'DAM': extract_geometry_data(dam),
                'RECT': extract_geometry_data(rect)
            })

        logger.info(f"{len(roi_list)}개의 ROI 데이터를 조회했습니다.")
        return roi_list

    except cx_Oracle.Error as error:
        logger.error(f"Oracle 오류 발생: {error}")
        return None
    except Exception as e:
        logger.error(f"예상치 못한 오류 발생: {e}")
        return None
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()
            
def mask_water(image):
    ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI')
    water_mask = ndwi.gt(0.0).selfMask()
    return water_mask

# Oracle SPATIAL Data 추가하기 테스트
## 기본 테스트 조건
- uploads 하위 디렉토리에 지역을 상징하는 디렉토리가 있어야 함
- 해당 디렉토리 내에는 dam.zip, polygon.zip, rect.zip 파일이 있어야 함
## 프로그램 동작
1. 프로그램 실행 시, uploads 하위 디렉토리 목록을 보여준다.
   1) 디렉토리 명1
   2) 디렉토리 명2
   ...
2. 사용자는 표준 입력으로 목록 중, 하나를 선택한다.(숫자 선택)
3. 디렉토리 경로를 전달받은 함수에서 아래 내용을 체크한다.
- 선택된 디렉토리 내에 3개의 파일 명이 정확히 존재하는지 체크한다.

In [ ]:
import zipfile
import shapefile

def check_directory_contents(directory_path):
    required_files = ['dam.zip', 'polygon.zip', 'rect.zip']
    files_in_directory = os.listdir(directory_path)
    
    missing_files = [file for file in required_files if file not in files_in_directory]
    
    if not missing_files:
        print("모든 필요한 파일이 존재합니다.")
        # 모든 파일의 전체 경로를 생성하여 반환
        return [os.path.join(directory_path, file) for file in required_files]
    else:
        print("다음 파일들이 누락되었습니다:", ", ".join(missing_files))
        return False

def main():
    uploads_dir = 'uploads'
    
    # uploads 디렉토리 내의 하위 디렉토리 목록 가져오기
    subdirectories = [d for d in os.listdir(uploads_dir) if os.path.isdir(os.path.join(uploads_dir, d))]
    print(f"subdirectories : {subdirectories}")
    if not subdirectories:
        logger.debug("uploads 디렉토리에 하위 디렉토리가 없습니다.")
        return
    
    # 하위 디렉토리 목록 출력
    print("uploads 하위 디렉토리 목록:")
    for i, directory in enumerate(subdirectories, 1):
        print(f"{i}) {directory}")
    
    # 사용자 입력 받기
    while True:
        logger.debug("사용자 입력 대기 중")
        try:
            choice = input("처리할 디렉토리 번호를 입력하세요 (종료하려면 'q'): ")
            if choice.lower() == 'q':
                logger.debug("사용자가 종료를 선택했습니다.")
                return
            
            choice = int(choice)
            if 1 <= choice <= len(subdirectories):
                selected_directory = subdirectories[choice - 1]
                logger.debug(f"선택된 디렉토리: {selected_directory}")
                break
            else:
                logger.debug(f"1에서 {len(subdirectories)} 사이의 숫자를 입력하세요.")
                print(f"1에서 {len(subdirectories)} 사이의 숫자를 입력하세요.")
        except ValueError:
            logger.debug("잘못된 입력")
            print("유효한 숫자나 'q'를 입력하세요.")
    
    # 선택된 디렉토리 경로
    selected_path = os.path.join(uploads_dir, selected_directory)
    
    # 디렉토리 내용 체크
    target_files = check_directory_contents(selected_path)
    
    # DB에 저장할 Shape 속성 추출
    shape_info_arr  = extract_coordinates(target_files)    
    persist_roi_shape_files(selected_directory, shape_info_arr)


    # DB에 저장
def persist_roi_shape_files(roi_nam, coordinates) : 
    print(f'{roi_nam} :  {coordinates}')
    try:
        # Oracle 연결
        connection = get_db_connection()
        cursor = connection.cursor()

        # 인덱스 비활성화
        cursor.execute("ALTER INDEX ROI_SPATIAL_IDX1 UNUSABLE")
        cursor.execute("ALTER INDEX ROI_SPATIAL_IDX2 UNUSABLE")
        cursor.execute("ALTER INDEX ROI_SPATIAL_IDX3 UNUSABLE")

        insert_query = f"""
        MERGE INTO ROI_SPATIAL target
        USING (
            SELECT :name AS roi_name, 
                SDO_GEOMETRY(
                    2002,  -- 2D LineString
                    4326,  -- SRID for WGS84
                    NULL,
                    SDO_ELEM_INFO_ARRAY(1, 2, 1),
                    SDO_ORDINATE_ARRAY({coordinates['dam_coordinates']})
                ) AS dam_geom,
                SDO_GEOMETRY(
                    2003,  -- 2D Polygon
                    4326,  -- SRID(Spatial Reference Identifier) for WGS84, WGS84와 매핑되는 값
                    NULL,
                    SDO_ELEM_INFO_ARRAY(1, 1003, 1),
                    SDO_ORDINATE_ARRAY({coordinates['polygon_coordinates']})
                ) AS polygon_geom,
                SDO_GEOMETRY(
                    2003,  -- 2D Polygon
                    4326,  -- SRID for WGS84
                    NULL,
                    SDO_ELEM_INFO_ARRAY(1, 1003, 1),
                    SDO_ORDINATE_ARRAY({coordinates['rect_coordinates']})
                ) AS rect_geom
            FROM dual
        ) source
        ON (target.ROI_NAME = source.roi_name)
        WHEN MATCHED THEN
            UPDATE SET target.DAM = source.dam_geom,
                    target.POLYGON = source.polygon_geom,
                    target.RECT = source.rect_geom
        WHEN NOT MATCHED THEN
            INSERT (ROI_ID, ROI_NAME, DAM, POLYGON, RECT)
            VALUES (MY_SHAPE_SEQ.NEXTVAL, source.roi_name, source.dam_geom, source.polygon_geom, source.rect_geom)
        """
        cursor.execute(insert_query, name=roi_nam)

        # 인덱스 활성화
        cursor.execute("ALTER INDEX ROI_SPATIAL_IDX1  REBUILD")
        cursor.execute("ALTER INDEX ROI_SPATIAL_IDX2  REBUILD")
        cursor.execute("ALTER INDEX ROI_SPATIAL_IDX3  REBUILD")

        # 인덱스 통계 수집
        cursor.execute("BEGIN DBMS_STATS.GATHER_INDEX_STATS(USER, 'ROI_SPATIAL_IDX1'); END;")
        cursor.execute("BEGIN DBMS_STATS.GATHER_INDEX_STATS(USER, 'ROI_SPATIAL_IDX2'); END;")
        cursor.execute("BEGIN DBMS_STATS.GATHER_INDEX_STATS(USER, 'ROI_SPATIAL_IDX3'); END;")

        return "Inserted features into the database."
    
    except cx_Oracle.Error as error:
        logger.debug(f"Oracle Error occurred: {error}")
        return f"Error: {error}"
    except Exception as e:
        logger.debug(f"An unexpected error occurred: {e}")
        return f"Error: {e}"
    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals():
            connection.close()

    
def extract_coordinates(target_files):
    print(target_files);
    result = {}
    
    for file_path in target_files:
        file_name = os.path.basename(file_path)
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # ZIP 파일의 내용을 임시 디렉토리에 추출
            temp_dir = file_name.replace('.zip', '_temp')
            zip_ref.extractall(temp_dir)
            
            # .shp 파일 찾기
            shp_file = next(f for f in os.listdir(temp_dir) if f.endswith('.shp'))
            shp_path = os.path.join(temp_dir, shp_file)
            
            # Shape 파일 읽기
            with shapefile.Reader(shp_path) as shp:
                # 첫 번째 shape의 점들을 가져옴
                shape = shp.shapes()[0]
                points = shape.points
                
                # 좌표를 문자열로 변환
                coordinates = ', '.join([f"{p[0]}, {p[1]}" for p in points])
                
                # 결과 저장
                if 'dam' in file_name.lower():
                    result['dam_coordinates'] = coordinates
                elif 'polygon' in file_name.lower():
                    result['polygon_coordinates'] = coordinates
                elif 'rect' in file_name.lower():
                    result['rect_coordinates'] = coordinates
            
            # 임시 디렉토리 삭제
            for f in os.listdir(temp_dir):
                os.remove(os.path.join(temp_dir, f))
            os.rmdir(temp_dir)
    
    return result

if __name__ == "__main__":
    main()

## GeoPandas 이용한 테스트

In [ ]:
import geopandas as gpd
import zipfile
import tempfile
from shapely.geometry import LineString, Polygon

def extract_coordinates_from_shapes(target_files):
    result = {}

    for file_path in target_files:
        file_name = os.path.basename(file_path)
        with tempfile.TemporaryDirectory() as temp_dir:
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                zip_ref.extractall(temp_dir)

            shp_file = next(f for f in os.listdir(temp_dir) if f.endswith('.shp'))
            shp_path = os.path.join(temp_dir, shp_file)

            gdf = gpd.read_file(shp_path)
            geometry = gdf.geometry.iloc[0]

            if isinstance(geometry, LineString):
                coordinates = ', '.join([f"{x}, {y}" for x, y in geometry.coords])
            elif isinstance(geometry, Polygon):
                coordinates = ', '.join([f"{x}, {y}" for x, y in geometry.exterior.coords])
            else:
                raise ValueError(f"Unsupported geometry type: {type(geometry)}")

            if 'dam' in file_name.lower():
                result['dam_coordinates'] = coordinates
            elif 'polygon' in file_name.lower():
                result['polygon_coordinates'] = coordinates
            elif 'rect' in file_name.lower():
                result['rect_coordinates'] = coordinates

    print(f'result : {result}')
          
    return result

if __name__ == "__main__":
    extract_coordinates_from_shapes(['uploads\\cuscos\\dam.zip', 'uploads\\cuscos\\polygon.zip', 'uploads\\cuscos\\rect.zip'])


# 배치 프로그램
## 기능 정의
- Shape 파일 정보 조회
- 최소거리/지표수 비율 계산하기
- 계산결과 저장하기
## 1. Shape 파일 정보 조회


In [ ]:
# 함수 사용 예시
if __name__ == "__main__":
    roi_data = get_roi_list()
    if roi_data:
        print(f"조회된 ROI 데이터 수: {len(roi_data)}")
        print(f"roi_data: {roi_data}")
        # 여기서 GEE 엔진을 통한 추가 작업을 수행할 수 있습니다.

    else:
        print("ROI 데이터 조회에 실패했습니다.")

## 2. 최소거리 계산하기

In [ ]:
import ee

def calculate_minimum_distance(roi_data):    
    logger.debug(f'roi_data : {roi_data}')
    # Earth Engine 초기화
    ee.Initialize()
    logger.debug('ee.Initialize() success')

    # ROI 데이터에서 dam과 polygon 추출
    dam_coordinates = roi_data['DAM']['SDO_ORDINATES']
    polygon_coordinates = roi_data['POLYGON']['SDO_ORDINATES']

    logger.debug(f'dam_coordinates : {dam_coordinates}')
    logger.debug(f'polygon_coordinates : {polygon_coordinates}')

    # dam LineString 생성
    dam_points = [[dam_coordinates[i], dam_coordinates[i+1]] for i in range(0, len(dam_coordinates), 2)]
    dam = ee.Geometry.LineString(dam_points)
    logger.debug(f'dam : {dam}')

    # polygon 생성
    polygon_points = [[polygon_coordinates[i], polygon_coordinates[i+1]] for i in range(0, len(polygon_coordinates), 2)]
    polygon = ee.Geometry.Polygon([polygon_points])
    logger.debug(f'polygon_points : {polygon_points}')

    # 최소 거리 계산
    distance = dam.distance(polygon)
    logger.debug(f'distance : {distance}')

    # 거리 값 가져오기
    distance_value = distance.getInfo()
    logger.debug(f'distance_value : {distance_value}')

    return distance_value

# 함수 사용 예시
if __name__ == "__main__":
    # ROI 데이터 예시 (get_roi_list 함수에서 반환된 형식을 가정)
    # ROI 데이터 가져오기
    roi_data = get_roi_list()
    
    if roi_data:
        for roi_item in roi_data:
            try:
                distance = calculate_minimum_distance(roi_item)
                print(f"ROI ID {roi_item['ROI_ID']}의 Dam과 Polygon 사이의 최소 거리: {distance} 미터")
            except Exception as e:
                logger.error(f"ROI ID {roi_item['ROI_ID']} 처리 중 오류 발생: {e}")
    else:
        print("ROI 데이터를 가져오는 데 실패했습니다.")

## 3. 물영역 감지하기
```javascript

var snippetId = 'COPERNICUS/S2_SR_HARMONIZED';
var startDate = '2024-01-01'; 
var endDate = '2024-06-30';     // today
var cloudCoverage = 20;
var imageCollection = ee.ImageCollection(snippetId)
    .filterBounds(polygon)
    .filterDate(startDate, endDate)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloudCoverage))
    .sort('system:time_start', false);
var dam = 
  ee.Geometry.LineString(
      [[127.10559951935767, 36.29340657585504],
       [127.10559951935767, 36.29190191317454],
       [127.10514890824317, 36.290846902494984],
       [127.10523473893164, 36.288529287963236],
       [127.10351812516211, 36.285398669494015],
       [127.10343229447363, 36.28470680316411]]);

var latestImage = imageCollection.first();

function maskWater(image) {
    var ndwi = image.normalizedDifference(['B3', 'B5']).rename('NDWI');
    var waterMask = ndwi.gt(0.0).selfMask();
    return waterMask;
}

function calculate(image) {
    var waterMask = maskWater(image);
    var vectors = waterMask.reduceToVectors({
        geometry: selectedImage.geometry(),
        scale: 30,
        eightConnected: false,
        maxPixels: 1e9,
        geometryType: 'polygon'
        });
    var polygon = vectors.geometry();

    var polygons = ee.FeatureCollection(polygon.coordinates().map(function(coords) {
        return ee.Feature(ee.Geometry.Polygon(coords));
    }));

    var polygonsWithVertexCount = polygons.map(function(feature) {
        var vertexCount = feature.geometry().coordinates().flatten().length();
        return feature.set('vertexCount', vertexCount);
    });

    var largestPolygon = polygonsWithVertexCount
        .sort('vertexCount', false)
        .first();

    // 최소 거리 계산
    var minDistance = dam.distance(largestPolygon, 1);
    return minDistance;

}

calculate(latestImage);

```


In [ ]:
from datetime import datetime
import geemap

logging.basicConfig(level=logging.INFO)

def calculate(roi_data):
    for roi in roi_data:
        snippet_id = 'COPERNICUS/S2_SR_HARMONIZED'
        start_date = '2024-01-01'
        end_date = '2024-06-30'  # today
        cloud_coverage = 20

        # ROI 데이터에서 polygon과 dam 추출
        polygon = ee.Geometry.Polygon(roi['POLYGON']['SDO_ORDINATES'])
        dam = ee.Geometry.LineString(roi['DAM']['SDO_ORDINATES'])
        
        # logger.debug(f'polygon : {polygon}');
        # logger.debug(f'dam : {dam}');
        

        image_collection = (ee.ImageCollection(snippet_id)
                            .filterBounds(polygon)
                            .filterDate(start_date, end_date)
                            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_coverage))
                            .sort('system:time_start', False)
                            .map(lambda image: image.clip(polygon)))  # 각 이미지를 polygon으로 클리핑

        latest_image = image_collection.first()
        time_start_ms = latest_image.get('system:time_start').getInfo()

        # Unix 타임스탬프(밀리초)를 datetime 객체로 변환
        time_start_datetime = datetime.fromtimestamp(time_start_ms / 1000)

        # datetime 객체를 'YYYY-MM-dd' 형식의 문자열로 변환
        time_start_formatted = time_start_datetime.strftime('%Y-%m-%d')

        logger.info(f"Latest image date: {time_start_formatted}")

        
        water_mask = mask_water(latest_image)
        # 지도 생성
        Map = geemap.Map()
        
        # 기본 이미지 추가
        vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
        Map.addLayer(latest_image, vis_params, 'Latest Image')
        
        # 물 마스크 추가
        Map.addLayer(water_mask, {'palette': ['blue']}, 'Water Mask')
        
        # ROI 추가
        Map.addLayer(polygon, {'color': 'red'}, 'ROI Polygon')
        Map.addLayer(dam, {'color': 'yellow'}, 'Dam')
        
        # 지도 중심 설정
        Map.centerObject(polygon, 10)
        
        # 지도 표시
        display(Map)

        vectors = water_mask.reduceToVectors(
            geometry=latest_image.geometry(),
            scale=30,
            eightConnected=False,
            maxPixels=1e9,
            geometryType='polygon'
        )

        polygon = vectors.geometry()
        polygons = ee.FeatureCollection(polygon.coordinates().map(lambda coords: ee.Feature(ee.Geometry.Polygon(coords))))

        polygons_with_vertex_count = polygons.map(lambda feature: feature.set('vertexCount', feature.geometry().coordinates().flatten().length()))

        largest_polygon = polygons_with_vertex_count.sort('vertexCount', False).first()

        # 최소 거리 계산
        min_distance = dam.distance(largest_polygon.geometry(), 1)

        # 결과 출력
        logger.info(f"ROI ID {roi['ROI_ID']}의 Dam과 가장 큰 물 polygon 사이의 최소 거리: {min_distance.getInfo()} 미터")

# 함수 사용 예시
if __name__ == "__main__":
    # ROI 데이터 가져오기
    roi_data = get_roi_list()
    calculate(roi_data)
    

INFO:__main__:Oracle에 연결 시도 중
DEBUG:__main__:연결 정보: localhost, 49161, xe
DEBUG:__main__:DSN: (DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=49161))(CONNECT_DATA=(SERVICE_NAME=xe)))
INFO:__main__:2개의 ROI 데이터를 조회했습니다.
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json


Latest image date: 2024-06-10


DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/maps?fields=name&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/maps?fields=name&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/maps?fields=name&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengin

Map(center=[36.290192716040806, 127.10253433530683], controls=(WidgetControl(options=['position', 'transparent…

DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json HTTP/1.1" 200 None
INFO:__main__:ROI ID 31141의 Dam과 가장 큰 물 polygon 사이의 최소 거리: 102.50218408836865 미터
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json


Latest image date: 2024-06-10


DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/maps?fields=name&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/maps?fields=name&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/maps?fields=name&alt=json HTTP/1.1" 200 None
DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/maps?fields=name&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengin

Map(center=[36.32836298558054, 127.16405352886702], controls=(WidgetControl(options=['position', 'transparent_…

DEBUG:googleapiclient.discovery:URL being requested: POST https://earthengine.googleapis.com/v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json
DEBUG:urllib3.connectionpool:https://earthengine.googleapis.com:443 "POST /v1/projects/earthengine-legacy/value:compute?prettyPrint=false&alt=json HTTP/1.1" 200 None
INFO:__main__:ROI ID 31142의 Dam과 가장 큰 물 polygon 사이의 최소 거리: 201.6422027257651 미터


DEBUG:Comm:handle_msg[c609af88484b4f2984736259cf384e03]({'header': {'date': datetime.datetime(2024, 7, 12, 8, 38, 13, 48000, tzinfo=tzutc()), 'msg_id': 'b5775db7-ab0b-4c4b-b5fe-e4bea0f29e7d', 'msg_type': 'comm_msg', 'session': '9507b14d-2685-40e1-917e-4eded8860e46', 'username': '67b23f4c-16c9-4d72-8d07-f0143e26e814', 'version': '5.2'}, 'msg_id': 'b5775db7-ab0b-4c4b-b5fe-e4bea0f29e7d', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': 'c609af88484b4f2984736259cf384e03', 'data': {'method': 'update', 'state': {'loading': False}, 'buffer_paths': []}}, 'buffers': []})
DEBUG:Comm:handle_msg[3da5adc801ea4241bfa2369db34fc7c4]({'header': {'date': datetime.datetime(2024, 7, 12, 8, 38, 13, 49000, tzinfo=tzutc()), 'msg_id': '4ed83e62-9c01-4823-bd7f-ce85c4d0385d', 'msg_type': 'comm_msg', 'session': '9507b14d-2685-40e1-917e-4eded8860e46', 'username': '67b23f4c-16c9-4d72-8d07-f0143e26e814', 'version': '5.2'}, 'msg_id': '4ed83e62-9c01-4823-bd7f-ce85c4d0385d', 'msg_typ